In [14]:
import pandas as pd
data = pd.read_csv('/workspaces/sheet1/sheet1.csv')
data.head()

,姓名,部门,1,2,3,4,5,6,7,8,...,22,23,24,25,26,27,28,29,30,31
0,员工1,房务部,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,员工2,房务部,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,员工3,房务部,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,08:29\n,NaN,NaN,NaN,NaN,08:34\n18:47,08:32\n,NaN
3,员工4,房务部,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,员工5,房务部,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
from pyecharts.globals import CurrentConfig, NotebookType
CurrentConfig.NOTEBOOK_TYPE = NotebookType.JUPYTER_LAB
from pyecharts import options as opts
from pyecharts.charts import Map, Timeline, Bar, Line, Pie
from pyecharts.components import Table
from pyecharts.options import ComponentTitleOpts

s = data.groupby(data['部门']).size()

pie = Pie()
pie.add('部门人数',[*s.items()])
pie.set_series_opts(label_opts=opts.LabelOpts(formatter="{b}:{c}人"))
pie.load_javascript()

In [26]:
pie.render_notebook()

In [27]:
# 定义函数，将打卡时间划分为3个时间段 6-9时、10-14时、17-20时，这是早、中、晚用餐时间段，员工打卡用餐


import datetime

def time_period(time_str):
    # 将时间字符串转换为datetime对象
    time = datetime.datetime.strptime(time_str, '%H:%M')

    # 提取小时数
    hour = time.hour

    # 判断时间属于哪个时间段
    if 6 <= hour < 9:
        c = '早餐'
    elif 10 <= hour < 14:
        c = '午餐'
    elif 17 <= hour < 20:
        c = '晚餐'
    else:
        c = '其他时间'

    return c


def sum_a_day(a):

    # 字典，存放早餐，中餐，晚餐的打卡次数
    result = {}
    
    # 逐行遍历
    for row in a.itertuples():
        for value in row[3:]:
            if(pd.notnull(value)):
                # 经过对打卡记录的单元进行split，产生一个列表，如['18:54', '     ']
                my_list = value.split('\n')
                # 对列表中空值进行处理
                time_str = next((i for i in my_list if i.strip()), None)
                # 用time_period函数，判断time_str是属于哪个时间段的，并根据返回值进行下一步处理
                match time_period(time_str):
                    case '早餐':
                        # print("早餐")
                        if '早餐' in result:
                            result['早餐'] = result['早餐'] + 1
                        else:
                            result['早餐'] = 1
                    case '午餐':
                        # print("午餐")
                        if '午餐' in result:
                            result['午餐'] = result['午餐'] + 1
                        else:
                            result['午餐'] = 1
                    case '晚餐':
                        # print("晚餐")
                        if '晚餐' in result:
                            result['晚餐'] = result['晚餐'] + 1
                        else:
                            result['晚餐'] = 1
                    case _:
                        print("无效条件")
    return result

In [28]:
# 通过分组不同的部门，然后对每一个groupby进行apply函数处理
# 提问：为什么apply中传给自定义函数的参数是一个dataf，答案是：groupby对象在使用apply()之后，不是groupby对象本身变成了DataFrame对象，而是apply()函数会将groupby对象中的每个分组作为一个DataFrame传递给自定义函数进行处理
# 所以这里分别将每一个部门分组中的数据，分别作为dataframe，作为参数传递给sum_a_day这个函数
#以下为统计各部门三餐用餐次数
departmen_count = data.groupby(data['部门']).apply(lambda x:sum_a_day(x))

# 进行排序
sum_values = departmen_count.apply(lambda x: sum(x.values()))
# 通过sort,并把sort后的index给departmen_count，相当于departmen_count[index1,index2...],把列顺序重塑
sorted_series = departmen_count[sum_values.sort_values(ascending=True).index]

/tmp/ipykernel_7352/483543109.py:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  departmen_count = data.groupby(data['部门']).apply(lambda x:sum_a_day(x))


In [29]:
bar=Bar()
bar.add_xaxis([*sorted_series.keys()])
bar.add_yaxis('早餐',[*sorted_series.apply(lambda x:x['早餐'])],label_opts=opts.LabelOpts(position='top'))
bar.add_yaxis('中餐',[*sorted_series.apply(lambda x:x['午餐'])],label_opts=opts.LabelOpts(position='top'))
bar.add_yaxis('晚餐',[*sorted_series.apply(lambda x:x['晚餐'])],label_opts=opts.LabelOpts(position='top'))
bar.add_yaxis('各部门餐费',[*sorted_series.apply(lambda x:x['早餐']*6+x['午餐']*12+x['晚餐']*12)],label_opts=opts.LabelOpts(position='top'))
bar.set_global_opts(title_opts=opts.TitleOpts("各部门用餐用餐次数"), xaxis_opts=opts.AxisOpts(axislabel_opts={"rotate": 45}))
bar.render_notebook()

## 每天用餐人数分析

In [30]:
def everyday_count_fun(a):
    # a是某一个单元格
    result = {}
    if(pd.notnull(a)):
        # 经过对打卡记录的单元进行split，产生一个列表，如['18:54', '     ']
        my_list = a.split('\n')
        # 对列表中空值进行处理
        time_str = next((i for i in my_list if i.strip()), None)
        # 用time_period函数，判断time_str是属于哪个时间段的，并根据返回值进行下一步处理
        match time_period(time_str):
            case '早餐':
                # print("早餐")
                if '早餐' in result:
                    result['早餐'] = result['早餐'] + 1
                else:
                    result['早餐'] = 1
            case '午餐':
                # print("午餐")
                if '午餐' in result:
                    result['午餐'] = result['午餐'] + 1
                else:
                    result['午餐'] = 1
            case '晚餐':
                # print("")
                if '晚餐' in result:
                    result['晚餐'] = result['晚餐'] + 1
                else:
                    result['晚餐'] = 1
            case '其他时间':
                # print("其他时间")
                if '其他时间' in result:
                    result['其他时间'] = result['其他时间'] + 1
                else:
                    result['其他时间'] = 1
            case _:
                print("无效条件")
    return result

t = {}
for day in data.columns[2:]:
    t[day] = {'早餐':0,'午餐':0,'晚餐':0}
    for cell in data[day]:
        result = everyday_count_fun(cell)
        if('早餐' in result):
            t[day]['早餐'] = t[day]['早餐'] + result['早餐']
        if('午餐' in result):
            t[day]['午餐'] = t[day]['午餐'] + result['午餐']
        if('晚餐' in result):
            t[day]['晚餐'] = t[day]['晚餐'] + result['晚餐']

print(t)   

{'1': {'早餐': 70, '午餐': 38, '晚餐': 59}, '2': {'早餐': 81, '午餐': 51, '晚餐': 79}, '3': {'早餐': 85, '午餐': 50, '晚餐': 75}, '4': {'早餐': 85, '午餐': 55, '晚餐': 100}, '5': {'早餐': 83, '午餐': 42, '晚餐': 58}, '6': {'早餐': 76, '午餐': 44, '晚餐': 69}, '7': {'早餐': 76, '午餐': 51, '晚餐': 57}, '8': {'早餐': 85, '午餐': 37, '晚餐': 64}, '9': {'早餐': 81, '午餐': 43, '晚餐': 81}, '10': {'早餐': 79, '午餐': 44, '晚餐': 70}, '11': {'早餐': 88, '午餐': 49, '晚餐': 93}, '12': {'早餐': 87, '午餐': 52, '晚餐': 73}, '13': {'早餐': 77, '午餐': 36, '晚餐': 86}, '14': {'早餐': 73, '午餐': 48, '晚餐': 80}, '15': {'早餐': 81, '午餐': 57, '晚餐': 71}, '16': {'早餐': 81, '午餐': 49, '晚餐': 84}, '17': {'早餐': 87, '午餐': 48, '晚餐': 74}, '18': {'早餐': 83, '午餐': 41, '晚餐': 90}, '19': {'早餐': 98, '午餐': 54, '晚餐': 81}, '20': {'早餐': 86, '午餐': 47, '晚餐': 88}, '21': {'早餐': 92, '午餐': 51, '晚餐': 82}, '22': {'早餐': 84, '午餐': 49, '晚餐': 72}, '23': {'早餐': 67, '午餐': 57, '晚餐': 73}, '24': {'早餐': 70, '午餐': 44, '晚餐': 62}, '25': {'早餐': 72, '午餐': 44, '晚餐': 71}, '26': {'早餐': 78, '午餐': 56, '晚餐': 75}, '27': {'早餐': 80, '午

In [31]:
bar=Bar()
bar.add_xaxis([*t.keys()])
bar.add_yaxis('早餐',[t[day]['早餐'] for day in t.keys()],label_opts=opts.LabelOpts(position='top'))
bar.add_yaxis('午餐',[t[day]['午餐'] for day in t.keys()],label_opts=opts.LabelOpts(position='top'))
bar.add_yaxis('晚餐',[t[day]['晚餐'] for day in t.keys()],label_opts=opts.LabelOpts(position='top'))
bar.set_global_opts(title_opts=opts.TitleOpts("当月每天用餐统计"), xaxis_opts=opts.AxisOpts(axislabel_opts={"rotate": 45}))
bar.render_notebook()

## 每天用餐高峰分析

In [32]:
import datetime


def round_minutes_to_ten(time_str):
    time = datetime.datetime.strptime(time_str, '%H:%M')
    # 计算以10为刻度的四舍五入值
    rounded_minutes = round(time.minute / 10) * 10
    new_hour = time.hour
    new_minute = rounded_minutes
    if new_minute == 60:
        new_hour += 1
        new_minute = 0
    elif new_minute > 60:
        new_hour += new_minute // 60
        new_minute = new_minute % 60
    # 构建新的时间对象
    new_time = time.replace(hour = new_hour, minute = new_minute)
    return str(new_time)[10:16]

def to_time_str(cell):
    item = []
    if type(cell)==str:
        for value in cell.split('\n'):
            value = value.strip()
            if value:
                # 对value进行四舍五入处理
                value = round_minutes_to_ten(value)
                item.append(value)
    return item

series_list = []    #使用一个series的list，合并每一列产生的series
for day in data.columns[2:]:
    result_series = data[day].apply(to_time_str)
    result_series = result_series[result_series.map(len)>0]
    result_series = result_series.explode()
    series_list.append(result_series)
combined_series = pd.concat(series_list)    # 合并所有列的series
group = combined_series.groupby(combined_series).count().to_dict()

In [33]:
bar=Bar()
bar.add_xaxis([*group.keys()])
bar.add_yaxis('用餐人数',[*group.values()],label_opts=opts.LabelOpts(position='top'))
bar.set_global_opts(title_opts=opts.TitleOpts("各时段用餐情况"), xaxis_opts=opts.AxisOpts(axislabel_opts={"rotate": 45}))
bar.render_notebook()

In [34]:
line=Line()
line.add_xaxis([*group.keys()])
line.add_yaxis('用餐人数',[*group.values()],label_opts=opts.LabelOpts(position='top'))
line.set_global_opts(title_opts=opts.TitleOpts("各时段用餐情况"), xaxis_opts=opts.AxisOpts(axislabel_opts={"rotate": 45}))
line.render_notebook()